In [162]:
import pandas as pd

In [163]:
df = pd.read_csv('takehome_users.csv', encoding='utf-8', encoding_errors='ignore')
dl = pd.read_csv('takehome_user_engagement.csv', encoding='utf-8', encoding_errors='ignore')
dl['time_stamp'] = pd.to_datetime(dl['time_stamp'])

assert not [x for x in dl['user_id'].unique() if x not in df['object_id'].values]

In [164]:
df = df.rename(columns={'object_id':'user_id'})

In [165]:
dl['user_id'].unique().shape

(8823,)

In [166]:
df['user_id'].unique().shape

(12000,)

In [167]:
dl['day'] = dl['time_stamp'].dt.floor('D')

pv = pd.pivot_table(
    dl.groupby(['user_id', 'day'])[['visited']].sum().reset_index(),
    index='day',
    columns='user_id',
    values='visited', 
    aggfunc='sum',
).fillna(0).sort_index()
assert pv.sum().sum() == dl['visited'].sum()
pv.head(5)

x = pd.DataFrame({'day':pd.date_range(pv.index.min(), pv.index.max(), freq='D')})
pv = x.merge(pv.reset_index(), on='day', how='left').fillna(0).set_index('day')
pv = (pv > 0).astype(int)
pv.head(5)

,1,2,3,4,5,6,7,10,11,13,...,11989,11990,11991,11992,11995,11996,11997,11998,11999,12000
day,,,,,,,,,,,,,,,,,,,,,
2012-05-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-06-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-06-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2012-06-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-06-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [168]:
x = (pv.rolling(7).sum().dropna() > 3).astype(int).sum()
adopted_user_ids = x[x > 0].index.tolist()
df['is_adopted'] = df['user_id'].isin(adopted_user_ids)

In [169]:
df['was_invited'] = df['invited_by_user_id'].notnull()

In [170]:
df.to_csv('takehome_users_prepped.csv', index=False)